In [3]:
!source /content/venv/bin/python3.11

/bin/bash: line 1: source: /content/venv/bin/python3.11: cannot execute binary file


In [4]:
!pip install -qU unsloth
!pip install -qU huggingface_hub hf_transfer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import os
from huggingface_hub import snapshot_download

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
snapshot_download(
    repo_id = "unsloth/gemma-3n-E4B-it-GGUF",
    local_dir = "unsloth/gemma-3n-E4B-it-GGUF",
    allow_patterns=["*UD-Q4_K_XL*","mmproj-BF16.gguf"]
)

'/content/unsloth/gemma-3n-E4B-it-GGUF'

In [13]:
from unsloth import FastModel
import torch

model_name = "unsloth/gemma-3n-E4B-it-bnb-4bit"
model, tokenizer = FastModel.from_pretrained(
    model_name = model_name,
    dtype = torch.bfloat16,
    max_seq_length = 1024,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.7.7: Fast Gemma3N patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
from transformers import TextStreamer
def inference(messages, max_new_tokens=128):
  return model.generate(
      **tokenizer.apply_chat_template(
          messages,
          add_generation_prompt = True,
          tokenize = True,
          return_dict = True,
          return_tensors = "pt"
      ).to("cuda"),
    max_new_tokens = max_new_tokens,
    temperature = 1.0,
    top_p = 0.95,
    top_k = 64,
   # streamer = TextStreamer(tokenizer,skip_prompt=True)
  )


In [16]:
messages = [
    {
        "role":"user",
        "content":[{"type":"text",
                    "text":"hi"
                  }]
    }
]

In [34]:
result = inference(messages)
result

tensor([[     2,    105,   2364,    107,   2202,    106,    107,    105,   4368,
            107,  10979,    993, 236888, 155818, 236743,    108,   3910,    740,
            564,   1601,    611,   3124, 236881,    138,   4602,    993,   4658,
            611, 236789, 236753,   1133,    531,   2910,   1003, 236764,    653,
            496,   2934,    611,    735, 236881,    564, 236789, 236757,   5508,
            531,  11184, 236888, 103453,    110,    106]], device='cuda:0')

In [35]:
result

tensor([[     2,    105,   2364,    107,   2202,    106,    107,    105,   4368,
            107,  10979,    993, 236888, 155818, 236743,    108,   3910,    740,
            564,   1601,    611,   3124, 236881,    138,   4602,    993,   4658,
            611, 236789, 236753,   1133,    531,   2910,   1003, 236764,    653,
            496,   2934,    611,    735, 236881,    564, 236789, 236757,   5508,
            531,  11184, 236888, 103453,    110,    106]], device='cuda:0')

In [37]:
tokens = result[0].tolist()

In [39]:
print(tokenizer.decode(tokens,skip_special_tokens = True))

user
hi
model
Hi there! 👋 

How can I help you today?  Is there anything you'd like to talk about, or a question you have? I'm ready to chat! 😊




